In [31]:
import os
import datetime
import calendar
import pandas as pd

ENTRY_NAMES = ['work', 'Work', 'Bar']  # Names used

workcal_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
rel_path = 'data/workfiles'
workfiles_path = os.path.join(workcal_path, rel_path)
filelist = os.listdir(workfiles_path)  # makes pathes independent of containing superfolder

worktimes_all = []  # stores all for all days the worktimes of a day in list

# todo: search all *.ics files via os
def delta2min(delta):
    return divmod(delta.seconds, 60)[0]

for filename in filelist:
    try:
        with open(os.path.join(workfiles_path, filename), 'r') as file:
            content = file.read()
            for possible_entry in ENTRY_NAMES:
                if possible_entry in content:
                    filelines = content.split('\n')
                    worktimes_all.append([x for x in filelines if 'Europe/Berlin:' in x])
    except:
        pass
# ['DTEND;TZID=Europe/Berlin:20191031T220000', 'DTSTART;TZID=Europe/Berlin:20191031T170000']

def parse(entry):
    kind, dtime = entry.split('Europe/Berlin:')
    dtime = datetime.datetime.strptime(dtime, '%Y%m%dT%H%M%S')
    kind = 'start' if 'start'.upper() in kind else 'end'
    return kind,dtime


worktimes_parsed = [[parse(entry)for entry in day]for day in worktimes_all if len(day) == 2]
workrange_shift = [(b[1] if a[0] == 'end' else a[1], a[1]-b[1] if a[0] == 'end' else b[1]-a[1]) for a, b in worktimes_parsed]
workrange_shift = [(dtime, delta2min(shiftrange)) for dtime,shiftrange in workrange_shift]
workrange_shift.sort()

df = pd.DataFrame(workrange_shift,columns=['datetime_shift','amount_min'])
df

,datetime_shift,amount_min
0,2010-12-10 18:00:00,240
1,2010-12-18 17:00:00,330
2,2018-10-13 10:30:00,240
3,2018-10-13 17:00:00,120
4,2018-10-14 10:30:00,240
...,...,...
89,2019-11-23 17:00:00,300
90,2019-11-24 11:00:00,225
91,2019-11-29 17:00:00,300
92,2019-11-30 10:30:00,255


In [32]:
import datetime
from datetime import date

dtime_amountminutes = workrange_shift
tday = date.today()
endoflastmon = tday - datetime.timedelta(days=tday.day)
startoflastmon = endoflastmon - datetime.timedelta(days=endoflastmon.day-1)
workedlastmonth = [ws for ws in dtime_amountminutes if startoflastmon <= ws[0].date() <= endoflastmon]

In [43]:
df = pd.DataFrame(workedlastmonth,columns=['datetime_shiftstart','shift_minutes'])
df

df['Arbeitsbeginn'] = [f'{dt.hour}:{dt.minute if dt.minute != 0 else "00"}' for dt in df['datetime_shiftstart']]
df['Tag'] = [dt.date().day for dt in df['datetime_shiftstart']]


df

,datetime_shiftstart,shift_minutes,Arbeitsbeginn,Tag
0,2019-11-01 17:00:00,300,17:00,1
1,2019-11-02 17:00:00,300,17:00,2
2,2019-11-03 11:00:00,180,11:00,3
3,2019-11-15 17:30:00,300,17:30,15
4,2019-11-16 10:30:00,255,10:30,16
5,2019-11-16 16:30:00,300,16:30,16
6,2019-11-17 11:00:00,225,11:00,17
7,2019-11-22 17:00:00,300,17:00,22
8,2019-11-23 17:00:00,300,17:00,23
9,2019-11-24 11:00:00,225,11:00,24
